In [1]:
import tweepy
import os.path as path
import numpy as np
import nltk.tokenize as tk
import math
import scipy.stats as stats
import pandas as pd
import huggingface_hub
from scipy.stats import pearsonr
from transformers import AutoTokenizer
from transformers import AutoModelForMaskedLM
import torch
from transformers import BertModel
import sklearn.cluster
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import random
import emoji

In [2]:
def get_all_examples():
    """Load all examples from a CSV file using pandas and return those that contain text.

    Returns:
        A list of tweets (or data) that contain text.
    """
    save_file = './twitter_emoji.csv'

    if path.exists(save_file):
        df = pd.read_csv(save_file, lineterminator = '\n')
        ids_to_examples = df[df['Text'].notna()].to_dict(orient='records')  # Convert to list of dicts
    else:
        print('Could not find tweets, so returning an empty list!')
        ids_to_examples = []

    return ids_to_examples

In [3]:
def get_tweets(raw_examples):
  tweets = []
  for tweet in raw_examples:
    for (key, value) in tweet.items():
      tweets.append(value)
  return tweets

In [4]:
import re

# Input tweet
tweet = """🧡@KeplerHomes AirdropBox event for #Arbitrum ecological users is here. A total of 550,000 addresses are eligible for #airdrop, and 5 types of AirDropbox with different scarcity can be issued.

💙Invitation code: 52DC39
🏆Airdrop Portal:👉 https://t.co/fudohu97uV"""

# gets rid of punctuation and https
def remove_gack(words):
  punctation = ".,/';:[]\-=`~!@#$%^&*()_+{}|<>?"
  for punc in punctation:
    while punc in words:
      words.remove(punc)
  while "https" in words:
    words.remove("https")
  return words


# Convert tweet into an array of words, keeping emojis
def tweet_to_words_with_emojis(tweet):
    # Match words, hashtags, mentions, URLs, numbers, and emojis
    words = re.findall(r'\w+|#\w+|@\w+|https?://\S+|[^\w\s]', tweet)
    words = remove_gack(words)
    return words

# Process the tweet
words_array = tweet_to_words_with_emojis(tweet)

# Print the result
print(words_array)


['🧡', '@KeplerHomes', 'AirdropBox', 'event', 'for', '#Arbitrum', 'ecological', 'users', 'is', 'here', 'A', 'total', 'of', '550', '000', 'addresses', 'are', 'eligible', 'for', '#airdrop', 'and', '5', 'types', 'of', 'AirDropbox', 'with', 'different', 'scarcity', 'can', 'be', 'issued', '💙', 'Invitation', 'code', '52DC39', '🏆', 'Airdrop', 'Portal', '👉', 't', 'co', 'fudohu97uV']


In [122]:
all_tweets = []
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/backhand_index_pointing_right.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/check_mark_button.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/check_mark.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/clown_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/cooking.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/egg.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/enraged_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/eyes.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/face_holding_back_tears.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/face_savoring_food.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/face_with_steam_from_nose.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/face_with_tears_of_joy.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/fearful_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/fire.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/folded_hands.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/ghost.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/grinning_face_with_sweat.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/hatching_chick.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/hot_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/loudly_crying_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/melting_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/middle_finger.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/party_popper.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/partying_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/pile_of_poo.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/rabbit_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/rabbit.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/red_heart.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/rolling_on_the_floor_laughing.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/saluting_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/skull.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/smiling_face_with_halo.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/smiling_face_with_heart-eyes.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/smiling_face_with_hearts.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/smiling_face_with_sunglasses.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/smiling_face_with_tear.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/smiling_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/sparkles.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/sun.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/thinking_face.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/thumbs_up.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/white_heart.csv", lineterminator='\n')["Text"]))
all_tweets.extend(list(pd.read_csv("emoji_twitter_dataset2/winking_face.csv", lineterminator='\n')["Text"]))

In [123]:
len(all_tweets)

860000

In [125]:
# raw_tweets = all_tweets
tweets = all_tweets #get_tweets(raw_tweets)
token_tweets = []
for tweet in tweets:
  token_tweets.append(tweet_to_words_with_emojis(tweet))

(token_tweets[:10])

[['🧡',
  '@KeplerHomes',
  'AirdropBox',
  'event',
  'for',
  '#Arbitrum',
  'ecological',
  'users',
  'is',
  'here',
  'A',
  'total',
  'of',
  '550',
  '000',
  'addresses',
  'are',
  'eligible',
  'for',
  '#airdrop',
  'and',
  '5',
  'types',
  'of',
  'AirDropbox',
  'with',
  'different',
  'scarcity',
  'can',
  'be',
  'issued',
  '💙',
  'Invitation',
  'code',
  '52DC39',
  '🏆',
  'Airdrop',
  'Portal',
  '👉',
  't',
  'co',
  'fudohu97uV'],
 ['Remember',
  'success',
  'in',
  'online',
  'business',
  'is',
  'a',
  'marathon',
  'not',
  'a',
  'sprint',
  'Keep',
  'at',
  'it',
  'stay',
  'focused',
  'and',
  'success',
  'will',
  'come',
  '"',
  '#patience',
  '#onlinebusiness',
  '#success',
  'For',
  'more',
  'tips',
  'and',
  'Strategies',
  'follow',
  'me',
  '👉',
  '@coach_lawrence1',
  't',
  'co',
  'IvtL9Om86J'],
 ['@occupied_9',
  '@Rhiannon_clare_',
  '@FightHaven',
  'Thanks',
  'for',
  'the',
  'update',
  'the',
  'sh',
  't',
  'country',
  '

In [ ]:
# all_emojis = []
# for tweet in tweets:
#     for em in emoji.emoji_list(tweet):
#         em = em["emoji"]
#         all_emojis.append(em)
# all_emojis = list(set(all_emojis))

In [ ]:
# tweets_array = [tweet_to_words_with_emojis(tweet) for tweet in tweets]
# tweets_array

[['🧡',
  '@KeplerHomes',
  'AirdropBox',
  'event',
  'for',
  '#Arbitrum',
  'ecological',
  'users',
  'is',
  'here',
  'A',
  'total',
  'of',
  '550',
  '000',
  'addresses',
  'are',
  'eligible',
  'for',
  '#airdrop',
  'and',
  '5',
  'types',
  'of',
  'AirDropbox',
  'with',
  'different',
  'scarcity',
  'can',
  'be',
  'issued',
  '💙',
  'Invitation',
  'code',
  '52DC39',
  '🏆',
  'Airdrop',
  'Portal',
  '👉',
  't',
  'co',
  'fudohu97uV'],
 ['Remember',
  'success',
  'in',
  'online',
  'business',
  'is',
  'a',
  'marathon',
  'not',
  'a',
  'sprint',
  'Keep',
  'at',
  'it',
  'stay',
  'focused',
  'and',
  'success',
  'will',
  'come',
  '"',
  '#patience',
  '#onlinebusiness',
  '#success',
  'For',
  'more',
  'tips',
  'and',
  'Strategies',
  'follow',
  'me',
  '👉',
  '@coach_lawrence1',
  't',
  'co',
  'IvtL9Om86J'],
 ['@occupied_9',
  '@Rhiannon_clare_',
  '@FightHaven',
  'Thanks',
  'for',
  'the',
  'update',
  'the',
  'sh',
  't',
  'country',
  '

In [ ]:
# all_emojis = []
# for tweet in tweets:
#     for em in emoji.emoji_list(tweet):
#         em = em["emoji"]
#         all_emojis.append(em)
# all_emojis = list(set(all_emojis))

In [126]:
all_emojis = []
for tweet in token_tweets:
    tokens = [token for token in tweet if emoji.is_emoji(token)]
    all_emojis.extend(tokens)
all_emojis = list(set(all_emojis))

In [127]:
from gensim.models import Word2Vec

model = Word2Vec(token_tweets, min_count=1, max_vocab_size=None)
model.build_vocab(token_tweets, update=True)

In [ ]:
model.train(token_tweets, total_examples=model.corpus_count, epochs=model.epochs, report_delay=1)

(2688847, 3259995)

In [130]:
model.wv[random.sample(all_emojis,k=1)]

array([[-0.05954113,  0.0477779 ,  0.09558113,  0.00266039,  0.03756942,
         0.08638659, -0.02272027,  0.07460319,  0.04920048, -0.02650371,
        -0.00828066, -0.0348882 ,  0.02140316,  0.00340551,  0.05900159,
        -0.02455767,  0.06280837, -0.07829682, -0.09476762, -0.11249544,
         0.05029468, -0.07320777,  0.0553268 , -0.0847626 , -0.03272368,
         0.08855694, -0.0599872 , -0.0236752 , -0.14605616,  0.0790734 ,
        -0.06574173,  0.08695899, -0.00266091, -0.10671137, -0.0280639 ,
         0.08131409, -0.03376193,  0.06619172, -0.02186091,  0.05099877,
         0.06690731,  0.01753296, -0.02571294,  0.09529445,  0.0415213 ,
        -0.01081796, -0.05748072,  0.02622535,  0.14751472,  0.04266597,
        -0.02869331, -0.04202088,  0.0583613 ,  0.07305824, -0.07801083,
         0.05182937,  0.0024576 , -0.00399932, -0.08950813,  0.01313502,
        -0.024092  , -0.00973289, -0.00665685, -0.01454901,  0.02404257,
        -0.06734772,  0.06278283,  0.02488649, -0.0

In [131]:
model.save("emoji2vec.model")